In [1]:
# Import libraries
from selenium import webdriver
from selenium.webdriver.support.select import Select
from selenium.webdriver.common.by import By
from selenium.common.exceptions import StaleElementReferenceException
from selenium.common.exceptions import NoSuchElementException
import numpy as np
import pandas as pd
import time

In [2]:
# Declare browser 
driver = webdriver.Chrome('chromedriver.exe') 
# Open url 
driver.get('https://www.amazon.com/ref=nav_logo')
driver.maximize_window()
time.sleep(2)

product_name = driver.find_element(By.XPATH,'//div[@class="nav-search-field "]/input')
product_name.click()
product_name.send_keys('Dove')
time.sleep(2)

search = driver.find_element(By.XPATH,'//div[@class="nav-right"]/div/span/input')
search.click()
time.sleep(2)

C:\Users\ADMIN\AppData\Local\Temp\ipykernel_13024\3426370776.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('chromedriver.exe')


### Crawl Dove's Price Data 

In [3]:
brands, price = [], []
brand = 'Dove'
for i in range(7):
    price_list = driver.find_elements(By.XPATH,'//div[@class="a-row a-size-base a-color-base"]/a/span[1]/span')
    
    for j in range(len(price_list)):
        brands.append(brand)
        price.append(price_list[j].text)
        
    try: 
        next = driver.find_element(By.XPATH,'//a[@class="s-pagination-item s-pagination-next s-pagination-button s-pagination-separator"]')
        next.click()
        time.sleep(5)
    except NoSuchElementException:
        pass

### Crawl Price Data for other Brands

In [4]:
brand_list = ['Sunsilk', 'Axe body', 'Rexona', 'Head and shoulders shampoo', 'Pantene shampoo', 'Old spice', 'Olay']
for k in range(len(brand_list)):
    driver.find_element(By.NAME,'field-keywords').clear()
    brand_search = driver.find_element(By.NAME,'field-keywords')
    brand_search.click()
    brand_search.send_keys(brand_list[k])
    search = driver.find_element(By.ID,'nav-search-submit-button')
    search.click()
    time.sleep(5)
    
    for i in range(7):
        price_list = driver.find_elements(By.XPATH,'//div[@class="a-row a-size-base a-color-base"]/a/span[1]')

        for j in range(len(price_list)):
            try: 
                brands.append(brand_list[k])
                price.append(price_list[j].text)
            except NoSuchElementException:
                pass 

        try: 
            next = driver.find_element(By.XPATH,'//a[@class="s-pagination-item s-pagination-next s-pagination-button s-pagination-separator"]')
            next.click()
            time.sleep(5)
        except NoSuchElementException:
            pass

In [5]:
# Create dataframe 
df = pd.DataFrame(list(zip(brands, price)),
                  columns=['Brand','Price'])
df.head()

,Brand,Price
0,Dove,
1,Dove,$15\n70
2,Dove,
3,Dove,$8\n97
4,Dove,


In [6]:
# Check data type 
df.dtypes

Brand    object
Price    object
dtype: object

In [7]:
# Replace data 
df['Price'] = df['Price'].str.replace('$','')
df['Price'] = df['Price'].str.replace('\n', '.')

# Change data type 
df['Price'] = pd.to_numeric(df['Price'], errors="coerce")

C:\Users\ADMIN\AppData\Local\Temp\ipykernel_13024\887882839.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['Price'] = df['Price'].str.replace('$','')


In [9]:
df.head()

,Brand,Price
0,Dove,NaN
1,Dove,15.70
2,Dove,NaN
3,Dove,8.97
4,Dove,NaN


In [10]:
# Checking null values 
df['Price'].isnull().sum()

344

In [11]:
# Drop null values 
df.dropna(subset=['Price'], inplace=True)

In [12]:
df.head()

,Brand,Price
1,Dove,15.70
3,Dove,8.97
5,Dove,32.37
7,Dove,14.49
9,Dove,27.96


In [13]:
# Export data 
df.to_excel('Price.xlsx',index=False)

I'm also use this method to crawl rating data 